In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from __future__ import print_function
from functools import reduce
import re
import tarfile

import numpy as np
import os as os

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential


from keras.layers import Dot
from keras.layers import Add
from keras.layers import Flatten
from keras.callbacks import ModelCheckpoint
from keras import regularizers


In [ ]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data
            if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    return (pad_sequences(xs, maxlen=story_maxlen),
            pad_sequences(xqs, maxlen=story_maxlen), np.array(ys))




In [ ]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 128
EPOCHS = 1000
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

try:
    path = get_file('babi-tasks-v1-2.tar.gz',
                    origin='https://s3.amazonaws.com/text-datasets/'
                           'babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2'
          '.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise


In [ ]:
# challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
with tarfile.open(path) as tar:
    train = get_stories(tar.extractfile(challenge.format('train')))
    test = get_stories(tar.extractfile(challenge.format('test')))
np.random.shuffle(train)
np.random.shuffle(test)
vocab = set()
for story, q, answer in train + test:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
story_maxlen = max(map(len, (x for x, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _ in train + test)))

x, xq, y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
tx, txq, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)

print('vocab = {}'.format(vocab))
print('x.shape = {}'.format(x.shape))
print('xq.shape = {}'.format(xq.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

In [ ]:
print("Building the embedding matrix...")
GLOVE_PATH = "../input/glove.6B.50d/"


f = open('../input/glove6b50d/glove.6B.50d.txt', 'r')
embeddings_index = {}
for line in f:
    values = line.split(" ")
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
    except ValueError:
        print(values[1:])
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((len(word_idx) + 1, EMBED_HIDDEN_SIZE))
for word, i in word_idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
print('Build model...')

question_Input = layers.Input(shape=xq[0].shape, name='question_Input')
story_Input = layers.Input(shape=x[0].shape, name='story_Input')

#Embed question
q_Embedding = layers.Embedding(input_dim = vocab_size, output_dim = EMBED_HIDDEN_SIZE, weights = [embedding_matrix], input_length = story_maxlen)(question_Input)
q_Encode = recurrent.GRU(EMBED_HIDDEN_SIZE, return_sequences=True, dropout=0.3)(q_Embedding)

#Embed story
s_Embedding = layers.Embedding(input_dim = vocab_size, output_dim = EMBED_HIDDEN_SIZE, weights = [embedding_matrix], input_length = story_maxlen)(story_Input)
s_Encode = recurrent.GRU(EMBED_HIDDEN_SIZE, return_sequences=True, dropout=0.3)(s_Embedding)

#Attention Layer
dot_merge = layers.Dot(axes=[1, 1])([q_Encode, s_Encode])
flat = Flatten()(dot_merge)
dense = layers.Dense(EMBED_HIDDEN_SIZE * story_maxlen, activation="sigmoid")(flat)
reshape = layers.Reshape((story_maxlen, EMBED_HIDDEN_SIZE))(dense)

#Final layers
sum_merge = layers.Concatenate()([q_Encode, reshape])
#sum_merge = layers.Add()([q_Encode, s_Encode])
flat2 = Flatten()(sum_merge)
dense3 = layers.Dense(EMBED_HIDDEN_SIZE, activation = "relu")(flat2)
dense2 = layers.Dense(vocab_size, activation = "softmax")(dense3)

model = Model(inputs=[question_Input, story_Input], outputs = [dense2])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# checkpoint
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

print('Model compiled.')

In [ ]:
print('Training')
print(model.summary())
model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.1)
loss, acc = model.evaluate([tx, txq], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

In [ ]:
y_pred = model.predict([tx, txq],
                           batch_size=BATCH_SIZE)

In [ ]:
y_pred_tar = y_pred
for i in range(len(y_pred_tar)):
    y_pred_tar[i][y_pred_tar[i] >= max(y_pred_tar[i])] = 1
    y_pred_tar[i][y_pred_tar[i] < max(y_pred_tar[i])] = 0
print(y_pred_tar[0])

In [ ]:
total = ty.shape[0]
EM = 0
for i, j in zip(ty, y_pred_tar):
    if np.argmax(i) == np.argmax(j):
        EM += 1
print(str(EM) + '/' + str(total))

In [ ]:
print(tx)
article = ''
question = ''
print(word_idx)
inv_word_idx = {v: k for k, v in word_idx.items()}
print(inv_word_idx)
for word in tx[6]:
    if word != 0:
        article += inv_word_idx[word] + ' '
for word in txq[6]:
    if word != 0:
        question += inv_word_idx[word] + ' '
print(article)
print(question)
print(inv_word_idx[np.argmax(ty[6])])
print(inv_word_idx[np.argmax(y_pred_tar[6])])
